In [1]:
import numpy as np
from scipy.ndimage import distance_transform_edt

# Set default random number seed
np.random.seed(1)

# Distance matrix
N = 11
NN = N * N

# Other latitude-longitude grid
ltd = np.linspace(0, 4, N)
lgd = np.linspace(0, 4, N)

# Transport weights
tt0 = 7.9
tt1 = 1

# No change in transport network
tau0 = np.full((N, N), tt0)
tau0[:, 5] = tt1
tau0[5, :] = tt1

tau1 = np.full((N, N), tt0)
tau1[:, 5] = tt1
tau1[5, :] = tt1

dist0 = np.zeros((NN, NN))
dist1 = np.zeros((NN, NN))

for z in range(NN):
    seed = np.zeros((N, N), dtype=bool)
    seed[z // N, z % N] = True
    temp = distance_transform_edt(~seed)
    dist0[z, :] = temp.flatten()
    temp = distance_transform_edt(~seed)
    dist1[z, :] = temp.flatten()

np.fill_diagonal(dist0, 1)
np.fill_diagonal(dist1, 1)
rdist = dist1 / dist0

# Define treatment
treat = np.zeros((N, N))
treat[:, 5] = 1
treat[5, :] = 1
treat = treat.flatten()

# Define east and west
Iwest = np.zeros((N, N))
Ieast = np.zeros((N, N))
Iwest[:, :7] = 1
Ieast[:, 7:] = 1
Iwest = Iwest.flatten()
Ieast = Ieast.flatten()

# Define distance weights
dopen = np.ones(dist0.shape)
dclosed = np.zeros(dist0.shape)
dclosed[Iwest == 1, :] = 1
dclosed[:, Ieast == 1] = 1

# Trade costs are a power function of effective distance
dist0 = dist0 ** 0.33
dist1 = dist1 ** 0.33

In [2]:
import numpy as np
import functions
from scipy.stats import gmean

# **************************;
# **** Parameterization ****;
# **************************;

# Share of goods in consumption expenditure (1-housing share);
alpha=0.75 
# Elasticity of substitution;
sigma=4
# Goods Frechet shape parameter;
theta=4 
# Worker Frechet shape parameter;
epsilon=3

param=[alpha, theta, epsilon]

# ***********************;
# **** Random shocks ****;
# ***********************;

a=np.random.normal(0,1,NN)
a=np.exp(a)
a[Iwest==1]=a[Iwest==1]/gmean(a[Iwest==1])
a[Ieast==1]=a[Ieast==1]/gmean(a[Ieast==1])

b=np.random.normal(0,1,NN)
b=np.exp(b)
b[Iwest==1]=b[Iwest==1]/gmean(b[Iwest==1])
b[Ieast==1]=b[Ieast==1]/gmean(b[Ieast==1])

print('Summary statistics a')
print([np.mean(a), np.std(a), np.max(a), np.min(a)])

print('Summary statistics b')
print([np.mean(b), np.std(b), np.max(b), np.min(b)])

# **************************;
# **** Other Parameters ****;
# **************************:

# Observations
nobs=NN
# Land area;
H=100*np.ones(nobs)
# Aggregate labor Supply;
LL=153889          # US civilian labor force 2010 (Statistical Abstract, millions);
LLwest=(np.sum(Iwest)/(np.sum(Iwest)+np.sum(Ieast)))*LL
LLeast=(np.sum(Ieast)/(np.sum(Iwest)+np.sum(Ieast)))*LL

Summary statistics a
[1.4422311651395312, 1.3793053042286845, 8.001727097132699, 0.09004401272663291]
Summary statistics b
[1.6465552047074161, 1.9893175563211023, 13.569316272825379, 0.09486007351612845]


In [3]:
from functions.solveabCtyClosed import solveabCtyClosed
from functions.solveLwCtyClosed import solveLwCtyClosed
from functions.pindex import pindex
from functions.landprice import landprice
from functions.expectut import expectut
from functions.welfare import welfare
from functions.realw import realw

# *********************;
# **** Trade Costs ****;
# *********************;

# *******************************************************;
# **** Closed Economy Solve for Endogenous Variables ****;
# *******************************************************;

fund = np.zeros((nobs, 5))
fund[:, 0] = a
fund[:, 1] = b
fund[:, 2] = H
fund[:, 3] = Iwest
fund[:, 4] = Ieast

# Solve for region populations and wages;
# Note: You need to define or import the function 'solveLwCtyClosed'
w, L, tradesh, dtradesh, Lconverge, wconverge, xtic = solveLwCtyClosed(param, fund, dclosed, dist0, nobs, LL, LLwest, LLeast)
print('>>>> Wage and Population System Converged <<<<')
print('>>>> Check Wage and Population Convergence <<<<')
print(wconverge, Lconverge)
print('>>>> Elapsed Time in Seconds <<<<')
print(xtic)

# Price index;
# Note: You need to define or import the function 'pindex'
P = pindex(param, fund, w, dtradesh, nobs, sigma)

# Land price;
# Note: You need to define or import the function 'landprice'
r = landprice(param, fund, L, w, dist0, nobs)

# Expected utility;
# Note: You need to define or import the function 'expectut'
EU = expectut(param, fund, L, w, tradesh, dist0, nobs, sigma)
print('>>>> Expected utility (West, East) <<<<')
print(np.unique(EU))

# Welfare;
# Note: You need to define or import the function 'welfare'
welf = welfare(param, fund, L, w, tradesh, dist0, nobs, sigma, LLeast, LLwest)
print('>>>> Welfare <<<<')
welf = np.round(welf * (10 ** 4))
welf = welf / (10 ** 4)
print(np.unique(welf))

# Real wage;
# Note: You need to define or import the function 'realw'
realwage = realw(param, fund, L, w, tradesh, dist0, nobs, sigma)

# ************************************************;
# **** Closed Economy Solve for Unobservables ****;
# ************************************************;

observe = np.zeros((nobs, 5))
observe[:, 0] = L
observe[:, 1] = w
observe[:, 2] = H
observe[:, 3] = Iwest
observe[:, 4] = Ieast

# Solve for region productivities and amenities;
# Note: You need to define or import the function 'solveabCtyClosed'
a_i, b_i, abtradesh, aconverge, bconverge, xtic = solveabCtyClosed(param, observe, dclosed, dist0, nobs, LLwest, LLeast)
print('>>>> Productivity and Amenity System Converged <<<<')
print('>>>> Check Productivity and Amenity Convergence <<<<')
print(aconverge, bconverge)
print('>>>> Elapsed Time in Seconds <<<<')
print(xtic)

>>>> Start Wage and Population Convergence <<<<
>>>> Wage and Population System Converged <<<<
>>>> Check Wage and Population Convergence <<<<
0 1
>>>> Elapsed Time in Seconds <<<<
1712116768.3359954
>>>> Expected utility (West, East) <<<<
[ 8.49900492 10.16158768]
>>>> Welfare <<<<
[ 8.4324 10.2419]
>>>> Start productivity and amenities Convergence <<<<
>>>> Productivity and Amenity System Converged <<<<
>>>> Check Productivity and Amenity Convergence <<<<
0 1
>>>> Elapsed Time in Seconds <<<<
0.08887195587158203


In [4]:
import numpy as np

# ***************************************************;
# **** CLOSED ECONOMY AGGREGATE TO COUNTRY LEVEL ****;
# ***************************************************;

NOBSC = 2

WBILL = np.zeros((NOBSC, 1))
WBILL[0, 0] = np.sum(w[Iwest == 1] * L[Iwest == 1])
WBILL[1, 0] = np.sum(w[Ieast == 1] * L[Ieast == 1])

OBSERVE = np.zeros((NOBSC, 5))
OBSERVE[0, 0] = np.sum(L[Iwest == 1])
OBSERVE[1, 0] = np.sum(L[Ieast == 1])
OBSERVE[0, 1] = WBILL[0, 0] / OBSERVE[0, 0]
OBSERVE[1, 1] = WBILL[1, 0] / OBSERVE[1, 0]
OBSERVE[0, 2] = np.sum(H[Iwest == 1])
OBSERVE[1, 2] = np.sum(H[Ieast == 1])
OBSERVE[0, 3] = 1
OBSERVE[1, 3] = 0
OBSERVE[0, 4] = 0
OBSERVE[1, 4] = 1

income = w * L
trade = tradesh * np.tile(income.T, (NN, 1))
TRADE = np.zeros((NOBSC, NOBSC))
TRADE[0, 0] = np.sum(trade[Iwest == 1, :][:, Iwest == 1])
TRADE[1, 1] = np.sum(trade[Ieast == 1, :][:, Ieast == 1])
TRADE[0, 1] = np.sum(trade[Iwest == 1, :][:, Ieast == 1])
TRADE[1, 0] = np.sum(trade[Ieast == 1, :][:, Iwest == 1])
EXPEND = np.sum(TRADE, axis=1)
TRADESH = TRADE / np.tile(EXPEND, (1, 2))
DTRADESH = np.diag(TRADESH)

# *****************************************************;
# **** Open Economy Solve for Endogenous Variables ****;
# *****************************************************;

# Solve for region populations and wages;
Cw, CL, Ctradesh, Cdtradesh, CLconverge, Cwconverge, Cxtic = solveLwCtyOpen(param, fund, dopen, dist1, nobs)
print('>>>> Wage and Population System Converged <<<<')
print('>>>> Check Wage and Population Convergence <<<<')
print(Cwconverge, CLconverge)
print('>>>> Elapsed Time in Seconds <<<<')
print(xtic)

# Counterfactual price index;
CP = pindex(param, fund, Cw, Cdtradesh, nobs)

# Counterfactual land prices;
Cr = landprice(param, fund, CL, Cw, dist1, nobs)

# Counterfactual expected utility;
CEU = expectut(param, fund, CL, Cw, Ctradesh, dist1, nobs)
print('>>>> Expected utility (West, East) <<<<')
print(np.unique(CEU))

# Counterfactual welfare;
Cwelf = welfare(param, fund, CL, Cw, Ctradesh, dist1, nobs)
print('>>>> Welfare <<<<')
Cwelf = np.round(Cwelf * (10 ** 4))
Cwelf = Cwelf / (10 ** 4)
print(np.unique(Cwelf))

# Counterfactual real wage;
Crealwage = realw(param, fund, CL, Cw, Ctradesh, dist1, nobs)

# Welfare gains;
welfgain = welfaregains(param, Ctradesh, tradesh, CL, L, nobs)
print('>>>> Welfare Gains <<<<')
welfgain = np.round(welfgain * (10 ** 4))
welfgain = welfgain / (10 ** 4)
print(np.unique(welfgain))

# Perfectly immobile welfare gains (ACR);
acrwelfgain = acrwelfaregains(param, Ctradesh, tradesh, CL, L, nobs)

# Perfectly mobile welfare gains;
mobwelfgain = mobwelfaregains(param, Ctradesh, tradesh, CL, L, nobs)

# ******************************************;
# **** Open Economy Solve Unobservables ****;
# ******************************************;

Cobserve = np.zeros((nobs, 5))
Cobserve[:, 0] = CL
Cobserve[:, 1] = Cw
Cobserve[:, 2] = H
Cobserve[:, 3] = Iwest
Cobserve[:, 4] = Ieast

# Solve for region productivities and amenities;
Ca_i, Cb_i, Cabtradesh, Caconverge, Cbconverge, Cxtic = solveabCtyOpen(param, Cobserve, dopen, dist1, nobs)
print('>>>> Productivity and Amenity System Converged <<<<')
print('>>>> Check Productivity and Amenity Convergence <<<<')
print(Caconverge, Cbconverge)
print('>>>> Elapsed Time in Seconds <<<<')
print(Cxtic)

ValueError: operands could not be broadcast together with shapes (2,2) (1,4) 

In [ ]:
import numpy as np
from scipy.stats.mstats import gmean

# AGGREGATE TO COUNTRY LEVEL
NOBSC = 2

CWBILL = np.zeros((NOBSC, 1))
CWBILL[0, 0] = np.sum(Cw[Iwest == 1] * CL[Iwest == 1])
CWBILL[1, 0] = np.sum(Cw[Ieast == 1] * CL[Ieast == 1])

COBSERVE = np.zeros((NOBSC, 5))
COBSERVE[0, 0] = np.sum(CL[Iwest == 1])
COBSERVE[1, 0] = np.sum(CL[Ieast == 1])
COBSERVE[0, 1] = CWBILL[0, 0] / COBSERVE[0, 0]
COBSERVE[1, 1] = CWBILL[1, 0] / COBSERVE[1, 0]
COBSERVE[0, 2] = np.sum(H[Iwest == 1])
COBSERVE[1, 2] = np.sum(H[Ieast == 1])
COBSERVE[0, 3] = 1
COBSERVE[1, 3] = 0
COBSERVE[0, 4] = 0
COBSERVE[1, 4] = 1

Cincome = Cw * CL
Ctrade = Ctradesh * np.tile(Cincome.T, (NN, 1))
CTRADE = np.zeros((NOBSC, NOBSC))
CTRADE[0, 0] = np.sum(Ctrade[Iwest == 1, :][:, Iwest == 1])
CTRADE[1, 1] = np.sum(Ctrade[Ieast == 1, :][:, Ieast == 1])
CTRADE[0, 1] = np.sum(Ctrade[Iwest == 1, :][:, Ieast == 1])
CTRADE[1, 0] = np.sum(Ctrade[Ieast == 1, :][:, Iwest == 1])
CEXPEND = np.sum(CTRADE, axis=1)
CTRADESH = CTRADE / np.tile(CEXPEND, (1, 2))
CDTRADESH = np.diag(CTRADESH)

# WELFARE GAINS
WELFGAIN = welfaregains(param, CTRADESH, TRADESH, COBSERVE[:, 0], OBSERVE[:, 0], NOBSC)
print('>>>> Aggregate Country Welfare Gains <<<<')
WELFGAIN = np.round(WELFGAIN * (10 ** 4))
WELFGAIN = WELFGAIN / (10 ** 4)
print(np.unique(WELFGAIN))

# PERFECTLY IMMOBILE WELFARE GAINS (ACR)
ACRWELFGAIN = acrwelfaregains(param, CTRADESH, TRADESH, COBSERVE[:, 0], OBSERVE[:, 0], NOBSC)

# PERFECTLY MOBILE WELFARE GAINS
MOBWELFGAIN = mobwelfaregains(param, CTRADESH, TRADESH, COBSERVE[:, 0], OBSERVE[:, 0], NOBSC)

# COMPARE REGION AND COUNTRY WELFARE GAINS
print('>>>> Region and Country Welfare Gains <<<<')
print('[Region Country]')
temp = [np.unique(welfgain[Iwest == 1]), np.unique(welfgain[Ieast == 1])]
print(np.column_stack((temp, WELFGAIN)))

# COUNTRY PERFECTLY IMMOBILE WELFARE GAINS
print('>>>> Country Perfectly Immobile Welfare Gains <<<<')
print('[Region Country]')
print(np.unique(ACRWELFGAIN))

# COUNTRY PERFECTLY MOBILE WELFARE GAINS
print('>>>> Region and Country Perfectly Mobile Welfare Gains <<<<')
print('[Region Country]')
print(np.unique(MOBWELFGAIN))

# Geometric Means
gdtradesh = np.zeros((NOBSC, 1))
gdtradesh[0, 0] = gmean(dtradesh[Iwest == 1])
gdtradesh[1, 0] = gmean(dtradesh[Ieast == 1])

Cgdtradesh = np.zeros((NOBSC, 1))
Cgdtradesh[0, 0] = gmean(Cdtradesh[Iwest == 1])
Cgdtradesh[1, 0] = gmean(Cdtradesh[Ieast == 1])

gL = np.zeros((NOBSC, 1))
gL[0, 0] = gmean(L[Iwest == 1])
gL[1, 0] = gmean(L[Ieast == 1])

gCL = np.zeros((NOBSC, 1))
gCL[0, 0] = gmean(CL[Iwest == 1])
gCL[1, 0] = gmean(CL[Ieast == 1])

test = ((gdtradesh / Cgdtradesh) ** (alpha / theta)) * ((gL / gCL) ** ((1 / epsilon) + (1 - alpha)))

print('>>> Compare Aggregate and Geometric Mean Domestic Trade Share')
print('[Aggregate Geometric]')
print(np.column_stack((1 / CDTRADESH, gdtradesh / Cgdtradesh)))

print('>>> Compare Aggregate and Geometric Mean Labor Supply')
print('[Aggregate Geometric]')
print(np.column_stack((np.ones((2, 1)), gL / gCL)))

print('>>>> Compare Region Welfare, Test and Aggregate Welfare <<<<')
print('[region test aggregate]')
temp = [np.unique(welfgain[Iwest == 1]), np.unique(welfgain[Ieast == 1])]
print(np.column_stack((temp, test, ACRWELFGAIN)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# PRODUCTIVITY
amat = np.reshape(a, (N, N))
XXL = np.arange(min(lgd), max(lgd), 0.001)
YYL = np.arange(min(ltd), max(ltd), 0.001)
ZZA = griddata((lgd, ltd), amat.flatten(), (XXL[None,:], YYL[:,None]), method='linear')

# AMENITIES
bmat = np.reshape(b, (N, N))
ZZB = griddata((lgd, ltd), bmat.flatten(), (XXL[None,:], YYL[:,None]), method='linear')

# POPULATION
Lmat = np.reshape(L, (N, N))
ZZL = griddata((lgd, ltd), Lmat.flatten(), (XXL[None,:], YYL[:,None]), method='linear')

# PRICE INDEX
Pmat = np.reshape(P, (N, N))
ZZP = griddata((lgd, ltd), Pmat.flatten(), (XXL[None,:], YYL[:,None]), method='linear')

# WAGE
wmat = np.reshape(w, (N, N))
ZZw = griddata((lgd, ltd), wmat.flatten(), (XXL[None,:], YYL[:,None]), method='linear')

# RELATIVE LAND PRICE
rmat = np.reshape(r, (N, N))
ZZr = griddata((lgd, ltd), rmat.flatten(), (XXL[None,:], YYL[:,None]), method='linear')

# MULTI-PANEL FIGURE
fig, axs = plt.subplots(3, 2)

# Productivity
axs[0, 0].contourf(XXL, YYL, ZZA, 5)
axs[0, 0].set_title('Panel A : Productivity')
axs[0, 0].set_xlabel('Longitude')
axs[0, 0].set_ylabel('Latitude')

# Amenities
axs[0, 1].contourf(XXL, YYL, ZZB, 5)
axs[0, 1].set_title('Panel B : Amenities')
axs[0, 1].set_xlabel('Longitude')
axs[0, 1].set_ylabel('Latitude')

# Population
axs[1, 0].contourf(XXL, YYL, ZZL, 5)
axs[1, 0].set_title('Panel C : Population')
axs[1, 0].set_xlabel('Longitude')
axs[1, 0].set_ylabel('Latitude')

# Price index
axs[1, 1].contourf(XXL, YYL, ZZP, 5)
axs[1, 1].set_title('Panel D : Price Index')
axs[1, 1].set_xlabel('Longitude')
axs[1, 1].set_ylabel('Latitude')

# Wages
axs[2, 0].contourf(XXL, YYL, ZZw, 5)
axs[2, 0].set_title('Panel E : Wages')
axs[2, 0].set_xlabel('Longitude')
axs[2, 0].set_ylabel('Latitude')

# Land prices
axs[2, 1].contourf(XXL, YYL, ZZr, 5)
axs[2, 1].set_title('Panel F : Land Prices')
axs[2, 1].set_xlabel('Longitude')
axs[2, 1].set_ylabel('Latitude')

plt.savefig('graphs/cnty_closed.pdf')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# Assuming the variables CL, L, Cw, w, Cr, r, CP, P, acrwelfgain, Crealwage, realwage, tradesh, Ctradesh, lgd, ltd, N, welfgain are already defined

dL = CL / L
ldL = np.log(dL)

dw = Cw / w
ldw = np.log(dw)

dr = Cr / r
ldr = np.log(dr)

dP = CP / P
ldP = np.log(dP)

lacrwelfgain = np.log(acrwelfgain)

drealw = Crealwage / realwage
ldrealw = np.log(drealw)

dtradesh = np.diag(tradesh)
Cdtradesh = np.diag(Ctradesh)
ddtradesh = Cdtradesh / dtradesh

# POPULATION
dLmat = dL.reshape(N, N)
XXL = np.arange(min(lgd), max(lgd), 0.001)
YYL = np.arange(min(ltd), max(ltd), 0.001)
ZZL = griddata((lgd, ltd), dLmat.flatten(), (XXL[None,:], YYL[:,None]), method='cubic')

# Repeat similar steps for other variables

# Plotting
fig, axs = plt.subplots(3, 2)

# Population
axs[0, 0].contourf(XXL, YYL, ZZL, 5)
axs[0, 0].set_xlabel('Longitude', fontsize=8)
axs[0, 0].set_ylabel('Latitude', fontsize=8)
axs[0, 0].set_title('Panel A : Population', fontsize=8)
axs[0, 0].tick_params(axis='both', which='major', labelsize=8)
axs[0, 0].axvline(x=2.6, color='black', linewidth=2)

# Repeat similar steps for other subplots

plt.savefig('graphs/cnty_open.pdf')